# Расчет вероятности оттока клиентов


In [2]:
from __future__ import division
import pandas as pd
import numpy as np

#pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 100)

churn_df = pd.read_csv('churn_utf.txt')
col_names = churn_df.columns.tolist()

print("Column names:")
print(col_names)

to_show = col_names[:6] + col_names[-6:]
print(to_show)
print("\nSample data:")
churn_df[to_show].head(6)

Column names:
['Штат', 'Длина счета', 'Код зоны', 'Номер телефона', 'Услуга международных звонков', 'Услуга VMail', 'Количество VMail', 'Дневные минуты', 'Дневные звонки', 'Дневная оплата', 'Вечерние минуты', 'Вечерние звонки', 'Вечерняя оплата', 'Ночные минуты', 'Ночные звонки', 'Ночная оплата', 'Международные минуты', 'Международные звонки', 'Международная оплата', 'Обращений в СЦ', 'Уход']
['Штат', 'Длина счета', 'Код зоны', 'Номер телефона', 'Услуга международных звонков', 'Услуга VMail', 'Ночная оплата', 'Международные минуты', 'Международные звонки', 'Международная оплата', 'Обращений в СЦ', 'Уход']

Sample data:


,Штат,Длина счета,Код зоны,Номер телефона,Услуга международных звонков,Услуга VMail,Ночная оплата,Международные минуты,Международные звонки,Международная оплата,Обращений в СЦ,Уход
0,KS,128,415,382-4657,False,True,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,False,True,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,False,False,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,True,False,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,True,False,8.41,10.1,3,2.73,3,False
5,AL,118,510,391-8027,True,False,9.18,6.3,6,1.70,0,False
